In [1]:
# This adds the path to import the development version (git repo) of NDI Python
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ndi import Experiment, DaqSystem, FileNavigator
from ndi.daqreaders import MockReader
from ndi.database import FileSystem as Database
from ndi import Query as Q
from ndi.database.utils import print_everything_in, destroy_everything_in

In [3]:
db = Database('../test_exp')
destroy_everything_in(db)

In [4]:
fn = FileNavigator(epoch_file_patterns=['.*\\.wav', '.*\\.txt'], 
                   metadata_file_pattern='.*\\.txt')

In [5]:
class MockEpochProbeMap: pass
ds = DaqSystem(name='myDaq1',
              file_navigator=fn,
              daq_reader=MockReader,
              epoch_probe_map=MockEpochProbeMap)
ds2 = DaqSystem(name='myDaq2',
               file_navigator=fn,
               daq_reader=MockReader,
              epoch_probe_map=MockEpochProbeMap)

In [6]:
e = Experiment(name='myExperiment3').connect(
    database=db,
    daq_systems=[ds, ds2]
)

In [7]:
e2doc = db.find_by_id(e.id)
e2 = Experiment.from_document(e2doc)

In [8]:
e2 == e

True

In [9]:
e2 == Experiment.from_document(
    db.find(Q('_metadata.name') == 'myExperiment3')[0]
)

True

In [10]:
db.find(Q('_metadata.name') == 'myExperiment2')

[]

In [11]:
is_daq_system = Q('_metadata.type') == DaqSystem.DOCUMENT_TYPE
# has_experiment_id = Q('_metadata.experiment_id') == e.id
ds_results = db.find(is_daq_system)

In [12]:
print(ds_results)

[<ndi.document.Document object at 0x10e23a9d0>, <ndi.document.Document object at 0x127171160>]


In [13]:
print(ds_results[0].data)
e.daq_systems[0] == DaqSystem.from_document(ds_results[0])

{'_metadata': {'name': 'myDaq1', 'type': 'ndi_daq_system', 'experiment_id': '9cd7a6f2af5347f3aa1da88a969f8de1', 'parent_id': '', 'asc_path': '', 'version_depth': 0, 'latest_version': True}, '_dependencies': {}, '_depends_on': [], 'epoch_ids': [], 'epoch_probe_map_class': 'MockEpochProbeMap', 'daq_reader_class': 'MockReader'}


False

In [14]:
db.find()

In [15]:
db.delete_by_id(e.id, force=True)

In [16]:
try:
    db.find_by_id(e.id)
except FileNotFoundError:
    print('File not found')

In [17]:
db.delete_many(force=True)

In [18]:
db.find()

[]

In [19]:
try:
    db.add(ds.document)
except FileExistsError as e:
    print(e)

In [20]:
db.upsert(ds.document, force=True)

In [21]:
print(ds.metadata['name'])

myDaq1


In [22]:
db.update_by_id(ds.id, { 'name': 'daq2' })

In [23]:
ds2 = db.find_by_id(ds.id)

In [24]:
print(ds2.metadata['name'])

myDaq1


In [25]:
try:
    db.add('a')
except TypeError as e:
    print(e)

Unexpected type <class 'str'>. Expected type <class 'ndi.document.Document'>.
